<a href="https://colab.research.google.com/github/mgozon/DLG-UROP/blob/main/DLG_Iris_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DLG - Iris Dataset
This notebook modifies the code in [Deep Leakage from Gradients](https://gist.github.com/Lyken17/91b81526a8245a028d4f85ccc9191884) to work with the Iris Dataset.

In [243]:
%matplotlib inline

import numpy as np
from pprint import pprint

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
import torchvision
from torchvision import models, datasets, transforms
torch.manual_seed(50)

print(torch.__version__, torchvision.__version__)

1.12.1+cu113 0.13.1+cu113


In [244]:
from sklearn.datasets import load_iris
dst = load_iris()

device = "cpu"
if torch.cuda.is_available():
    device = "cuda"
print("Running on %s" % device)

def label_to_onehot(target, num_classes = 3):
    target = torch.unsqueeze(target, 1)
    onehot_target = torch.zeros(target.size(0), num_classes, device=target.device)
    onehot_target.scatter_(1, target, 1)
    return onehot_target

def cross_entropy_for_onehot(pred, target):
    return torch.mean(torch.sum(- target * F.log_softmax(pred, dim=-1), 1))

Running on cuda


In [245]:
def weights_init(m):
    if hasattr(m, "weight"):
        m.weight.data.uniform_(-0.5, 0.5)
    if hasattr(m, "bias"):
        m.bias.data.uniform_(-0.5, 0.5)
    
# architecture of NN is a fully connected neural network
# f(x) = \sigma_L(A_L \sigma (A_{L-1} (... (\sigma (A_{1} x + b) + ...  )+b_{L-1})+b_L)
class FcNet(nn.Module):
    def __init__(self):
        super(FcNet, self).__init__()
        act = nn.Sigmoid
        self.body = nn.Sequential(
            nn.Linear(4, 100),
            act(),
            nn.Linear(100, 100),
            act(),
            nn.Linear(100, 100),
            act(),
            nn.Linear(100, 3),
            act(),
        )
        
    def forward(self, x):
        out = self.body(x)
        out = out.view(out.size(0), -1)
        return out
    
net = FcNet().to(device)
    
net.apply(weights_init)
criterion = cross_entropy_for_onehot

In [246]:
######### honest partipant #########
from random import randint
flower_index = randint(0, 149)

gt_data = torch.tensor(dst.data[flower_index, :]).to(device)
gt_data = gt_data.view(1, *gt_data.size())
gt_label = torch.tensor(dst.target[flower_index]).to(device)
gt_label = gt_label.view(1)
gt_onehot_label = label_to_onehot(gt_label, num_classes=3)

# print out (data, label) and verify onehot
print(f"gt_data: {gt_data}")
print(f"gt_label: {gt_label}")
print(f"gt_onehot_label: {gt_onehot_label}")
print(f"flower {flower_index} has label (gt, onehot) = ({gt_label.item()}, {torch.argmax(gt_onehot_label, dim=-1).item()})")

# compute original gradient 
out = net(gt_data.float())
y = criterion(out, gt_onehot_label)
dy_dx = torch.autograd.grad(y, net.parameters())

# share the gradients with other clients
original_dy_dx = list((_.detach().clone() for _ in dy_dx))

gt_data: tensor([[5.6000, 3.0000, 4.1000, 1.3000]], device='cuda:0',
       dtype=torch.float64)
gt_label: tensor([1], device='cuda:0')
gt_onehot_label: tensor([[0., 1., 0.]], device='cuda:0')
flower 88 has label (gt, onehot) = (1, 1)


In [247]:
# generate dummy data and label
dummy_data = torch.randn(gt_data.size()).to(device).requires_grad_(True)
dummy_label = torch.randn(gt_onehot_label.size()).to(device).requires_grad_(True)

print("Dummy label is %d." % torch.argmax(dummy_label, dim=-1).item())

Dummy label is 2.


In [248]:
# identify (data, label) using gradient descent on the squared difference between the original and guessed gradient
optimizer = torch.optim.LBFGS([dummy_data, dummy_label] )

for iters in range(5):
    def closure():
        optimizer.zero_grad()

        pred = net(dummy_data) 
        print(f"prediction: {pred} from data: {dummy_data.data} and label: {dummy_label}")
        dummy_onehot_label = F.softmax(dummy_label, dim=-1)
        dummy_loss = criterion(pred, dummy_onehot_label) # TODO: fix the gt_label to dummy_label in both code and slides.
        dummy_dy_dx = torch.autograd.grad(dummy_loss, net.parameters(), create_graph=True)
        
        grad_diff = 0
        grad_count = 0
        for gx, gy in zip(dummy_dy_dx, original_dy_dx): # TODO: fix the variablas here
            grad_diff += ((gx - gy) ** 2).sum()
            grad_count += gx.nelement()
        # grad_diff = grad_diff / grad_count * 1000
        grad_diff.backward()
        
        return grad_diff
    
    optimizer.step(closure)
    current_loss = closure()
    print(iters, "%.4f" % current_loss.item())
        

prediction: tensor([[0.3383, 0.7384, 0.3646]], device='cuda:0', grad_fn=<ViewBackward0>) from data: tensor([[ 0.6411,  0.6207, -0.9301, -1.5124]], device='cuda:0') and label: tensor([[-2.1449, -1.0588, -0.9202]], device='cuda:0', requires_grad=True)
prediction: tensor([[0.3383, 0.7381, 0.3647]], device='cuda:0', grad_fn=<ViewBackward0>) from data: tensor([[ 0.6451,  0.6096, -0.9308, -1.5087]], device='cuda:0') and label: tensor([[-2.1794, -0.5685, -1.3759]], device='cuda:0', requires_grad=True)
prediction: tensor([[0.3385, 0.7360, 0.3670]], device='cuda:0', grad_fn=<ViewBackward0>) from data: tensor([[ 0.7057,  0.5408, -0.8855, -1.4733]], device='cuda:0') and label: tensor([[-2.4976,  0.3149, -1.9412]], device='cuda:0', requires_grad=True)
prediction: tensor([[0.3385, 0.7344, 0.3693]], device='cuda:0', grad_fn=<ViewBackward0>) from data: tensor([[ 0.7745,  0.5056, -0.8203, -1.4341]], device='cuda:0') and label: tensor([[-2.5987,  0.5456, -2.0707]], device='cuda:0', requires_grad=True)


In [249]:
# compare results
print(f"Original data: {gt_data}")
print(f"Predicted data: {dummy_data}")
print(f"Original label: {gt_label.item()}")
print(f"Predicted label: {torch.argmax(dummy_label).item()}")
print(f"Label SE: {((gt_data - dummy_data)**2).sum()}");

Original data: tensor([[5.6000, 3.0000, 4.1000, 1.3000]], device='cuda:0',
       dtype=torch.float64)
Predicted data: tensor([[5.6001, 2.9999, 4.1001, 1.3001]], device='cuda:0', requires_grad=True)
Original label: 1
Predicted label: 1
Label SE: 5.050050106105812e-08
